In [28]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

# from attention_utils import get_activations, get_data_recurrent


In [29]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [30]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [63]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [82]:
def sequential_generator(filename, batch_size):
    
    f = open(filename)
    
#     file_length = sum(1 for line in open(filename, encoding = 'UTF-8'))
#     shuffled_indexes = range(1, file_length + 1)
    # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#     index_position = 0
    
    

    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#     batch_features_lg = np.zeros((batch_size, nb_sequence_length, nb_embedding2_dims))
    # batch_features_idx = np.zeros((batch_size, nb_sequence_length))
        batch_labels = np.zeros((batch_size, 2))
        # print(len(features))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
#             line = linecache.getline(filename, shuffled_indexes[index_position])
            data = line.strip().split('\t')
            if len(data)!=2:
                i-=1
                continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            # print(batch_features_ft[i])
            # print(batch_features_ft[i].shape)
            batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
#             index_position += 1
#             if index_position == file_length:
                # shuffle indexes again
#                 shuffled_indexes = range(1, file_length + 1)
                # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#                 index_position = 0
#                 break
        # yield [batch_features_ft, batch_features_lg], batch_labels
        yield ([batch_features_ft], batch_labels)

In [65]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims):
    words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [37]:
f = open('/home/jindal/notebooks/jindal/NER/language_model/FB_counter-speech.csv')

line = f.readline()


In [46]:
f = open('/home/jindal/notebooks/jindal/NER/language_model/FB_counter-speech.csv')

for line in f:
    line = line.strip().split('\t')
    
    try:
        assert len(line)==2
    except AssertionError as e:
        print(line)
        print(len(line))

['Islam ist KEINE Religion', '', ' Der islam mit der dazugehörigen scharia ist eine faschistische und unmenschliche Ideologie (ein allumfassendes Regelwerk für das gesamte Leben sozial, kulturell, militärisch, politisch, rechtlich und religiös) um einen barbarischen, pädophilen Propheten, einem Massenmörder und Sklaventreiber der die Welt seit Jahrhunderten unterdrücken und beherrschen will!', 'OFFENSE']
4
['Islam ist KEINE Religion', '', ' Der islam mit der dazugehörigen scharia ist eine faschistische und unmenschliche Ideologie (ein allumfassendes Regelwerk für das gesamte Leben sozial, kulturell, militärisch, politisch, rechtlich und religiös) um einen barbarischen, pädophilen Propheten, einem Massenmörder und Sklaventreiber der die Welt seit Jahrhunderten unterdrücken und beherrschen will!', 'OFFENSE']
4
['OTHER']
1


In [71]:
train_lines = [line.strip().split("\t") for line in open('/home/jindal/notebooks/jindal/NER/language_model/FB_train.csv') if len(line.strip().split('\t'))==2]
dev_lines = [line.strip().split("\t") for line in open('/home/jindal/notebooks/jindal/NER/language_model/FB_dev.csv') if len(line.strip().split())==2]

In [92]:
train_sentences = [x[0] for x in train_lines]
train_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in train_lines])
train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]

dev_sentences = [x[0] for x in dev_lines]
dev_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in dev_lines])
dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]

In [73]:
n_labels =2

In [87]:
model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
lstm_block = LeakyReLU()(lstm_block)

filter_sizes = (3, 4, 5)
conv_blocks = []
for sz in filter_sizes:
    conv = Conv1D(
        filters = 200,
        kernel_size = sz,
        padding = 'valid',
        strides = 1
    )(lstm_block)
    conv = LeakyReLU()(conv)
    conv = GlobalMaxPooling1D()(conv)
    conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
# model_concatenated = Dropout(0.8)(model_concatenated)
model_concatenated = Dense(100)(model_concatenated)
model_concatenated = LeakyReLU()(model_concatenated)
model_output = Dense(n_labels, activation = "softmax")(model_concatenated)
model = Model(model_input_embedding, model_output)
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 75, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 75, 200)      320800      input_5[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_21 (LeakyReLU)      (None, 75, 200)      0           bidirectional_5[0][0]            
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 73, 200)      120200      leaky_re_lu_21[0][0]             
__________________________________________________________________________________________________
conv1d_14 

In [88]:
samples_per_epoch = len(train_sentences)
epochs = 50
batch_size = 32
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
checkpoint = ModelCheckpoint('best_classification_model_FB.h5', 
                             monitor='val_acc', 
                             verbose = 1, 
                             save_best_only = True, 
                             save_weights_only = True)

In [89]:
model.reset_states()
model.fit_generator(
    sequential_generator('/home/jindal/notebooks/jindal/NER/language_model/FB_train.csv', batch_size), 
    steps_per_epoch=steps_per_epoch, epochs=epochs,
    validation_data = sequential_generator('/home/jindal/notebooks/jindal/NER/language_model/FB_dev.csv', batch_size),
    validation_steps = math.ceil(len(dev_sentences) / batch_size),
    callbacks = [checkpoint]
)

Epoch 1/50
132/132 [==============================] - 33s 249ms/step - loss: 0.5878 - acc: 0.7102 - val_loss: 0.5886 - val_acc: 0.6562

Epoch 00001: val_acc improved from -inf to 0.65625, saving model to best_classification_model_FB.h5
Epoch 2/50
132/132 [==============================] - 31s 233ms/step - loss: 0.5493 - acc: 0.7320 - val_loss: 0.5568 - val_acc: 0.7188

Epoch 00002: val_acc improved from 0.65625 to 0.71875, saving model to best_classification_model_FB.h5
Epoch 3/50
132/132 [==============================] - 30s 230ms/step - loss: 0.5107 - acc: 0.7547 - val_loss: 0.5029 - val_acc: 0.8438

Epoch 00003: val_acc improved from 0.71875 to 0.84375, saving model to best_classification_model_FB.h5
Epoch 4/50
132/132 [==============================] - 30s 229ms/step - loss: 0.4890 - acc: 0.7723 - val_loss: 0.4061 - val_acc: 0.8125

Epoch 00004: val_acc did not improve from 0.84375
Epoch 5/50
132/132 [==============================] - 31s 232ms/step - loss: 0.4603 - acc: 0.7817 - 

# TRANSFER LEARNING HERE


In [113]:
n_labels =2

In [120]:
model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
lstm_block = LeakyReLU()(lstm_block)

filter_sizes = (3, 4, 5)
conv_blocks = []
for sz in filter_sizes:
    conv = Conv1D(
        filters = 200,
        kernel_size = sz,
        padding = 'valid',
        strides = 1
    )(lstm_block)
    conv = LeakyReLU()(conv)
    conv = GlobalMaxPooling1D()(conv)
    conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
# model_concatenated = Dropout(0.8)(model_concatenated)
model_concatenated = Dense(100)(model_concatenated)
model_concatenated = LeakyReLU()(model_concatenated)
model_output = Dense(n_labels, activation = "softmax")(model_concatenated)
new_model = Model(model_input_embedding, model_output)
new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
new_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 75, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_9 (Bidirectional) (None, 75, 200)      320800      input_9[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_41 (LeakyReLU)      (None, 75, 200)      0           bidirectional_9[0][0]            
__________________________________________________________________________________________________
conv1d_25 (Conv1D)              (None, 73, 200)      120200      leaky_re_lu_41[0][0]             
__________________________________________________________________________________________________
conv1d_26 

In [121]:
new_model.load_weights('best_classification_model_FB.h5')

In [122]:
train_lines = [line.strip().split("\t") for line in open('/home/gwiedemann/notebooks/OffLang/sample_train.txt')]
dev_lines = [line.strip().split("\t") for line in open('/home/gwiedemann/notebooks/OffLang/sample_dev.txt')]

In [123]:
train_sentences = [x[0] for x in train_lines]
train_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in train_lines])
train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]

dev_sentences = [x[0] for x in dev_lines]
dev_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in dev_lines])
dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]

In [124]:
def sequential_generator_germeval(filename, batch_size):
    
    f = open(filename)
    
#     file_length = sum(1 for line in open(filename, encoding = 'UTF-8'))
#     shuffled_indexes = range(1, file_length + 1)
    # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#     index_position = 0
    
    

    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#     batch_features_lg = np.zeros((batch_size, nb_sequence_length, nb_embedding2_dims))
    # batch_features_idx = np.zeros((batch_size, nb_sequence_length))
        batch_labels = np.zeros((batch_size, 2))
        # print(len(features))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
#             line = linecache.getline(filename, shuffled_indexes[index_position])
            data = line.strip().split('\t')
            if len(data)!=3:
                i-=1
                continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            # print(batch_features_ft[i])
            # print(batch_features_ft[i].shape)
            batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
#             index_position += 1
#             if index_position == file_length:
                # shuffle indexes again
#                 shuffled_indexes = range(1, file_length + 1)
                # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#                 index_position = 0
#                 break
        # yield [batch_features_ft, batch_features_lg], batch_labels
        yield ([batch_features_ft], batch_labels)

In [125]:
samples_per_epoch = len(train_sentences)
epochs = 50
batch_size = 32
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
checkpoint = ModelCheckpoint('best_classification_model.h5', 
                             monitor='val_acc', 
                             verbose = 1, 
                             save_best_only = True, 
                             save_weights_only = True)

In [126]:

new_model.fit_generator(
    sequential_generator_germeval('/home/gwiedemann/notebooks/OffLang/sample_train.txt', batch_size), 
    steps_per_epoch=steps_per_epoch, epochs=epochs,
    validation_data = sequential_generator_germeval('/home/gwiedemann/notebooks/OffLang/sample_dev.txt', batch_size),
    validation_steps = math.ceil(len(dev_sentences) / batch_size),
    callbacks = [checkpoint]
)

Epoch 1/50
132/132 [==============================] - 35s 266ms/step - loss: 0.7157 - acc: 0.7150 - val_loss: 0.5108 - val_acc: 0.7548

Epoch 00001: val_acc improved from -inf to 0.75481, saving model to best_classification_model.h5
Epoch 2/50
132/132 [==============================] - 32s 243ms/step - loss: 0.5006 - acc: 0.7464 - val_loss: 0.4754 - val_acc: 0.7656

Epoch 00002: val_acc improved from 0.75481 to 0.76562, saving model to best_classification_model.h5
Epoch 3/50
132/132 [==============================] - 32s 244ms/step - loss: 0.4623 - acc: 0.7848 - val_loss: 0.4762 - val_acc: 0.7933

Epoch 00003: val_acc improved from 0.76562 to 0.79327, saving model to best_classification_model.h5
Epoch 4/50
132/132 [==============================] - 32s 242ms/step - loss: 0.4255 - acc: 0.7985 - val_loss: 0.4702 - val_acc: 0.7873

Epoch 00004: val_acc did not improve from 0.79327
Epoch 5/50
132/132 [==============================] - 32s 246ms/step - loss: 0.3986 - acc: 0.8227 - val_loss:

132/132 [==============================] - 32s 246ms/step - loss: 0.0439 - acc: 0.9837 - val_loss: 1.2138 - val_acc: 0.7969

Epoch 00043: val_acc did not improve from 0.81611
Epoch 44/50
132/132 [==============================] - 32s 244ms/step - loss: 0.0570 - acc: 0.9777 - val_loss: 0.9426 - val_acc: 0.8053

Epoch 00044: val_acc did not improve from 0.81611
Epoch 45/50
132/132 [==============================] - 32s 240ms/step - loss: 0.0358 - acc: 0.9865 - val_loss: 1.1137 - val_acc: 0.8161

Epoch 00045: val_acc did not improve from 0.81611
Epoch 46/50
132/132 [==============================] - 32s 243ms/step - loss: 0.0286 - acc: 0.9884 - val_loss: 1.1234 - val_acc: 0.8161

Epoch 00046: val_acc did not improve from 0.81611
Epoch 47/50
132/132 [==============================] - 32s 245ms/step - loss: 0.0420 - acc: 0.9848 - val_loss: 1.0256 - val_acc: 0.8089

Epoch 00047: val_acc did not improve from 0.81611
Epoch 48/50
132/132 [==============================] - 32s 246ms/step - loss:

In [127]:
testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
for i in range(len(dev_sentences)):
    testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
results = new_model.predict(testset_features)

In [128]:
idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
predLabels = results.argmax(axis=-1)
devLabels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]
# print(idx2Label)
# print(predLabels)
# print(devLabels)
f1 = f1_score(devLabels, predLabels, average='binary', pos_label=1)
r = recall_score(devLabels, predLabels, average='binary', pos_label=1)
p = precision_score(devLabels, predLabels, average='binary', pos_label=1)
a = accuracy_score(devLabels, predLabels)
print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))

Test-Data: Prec: 0.690, Rec: 0.715, F1: 0.702, Acc: 0.800
